In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define the path in Google Drive
file_path = '/content/drive/MyDrive/MS AI/F2024/CMPE 295B/Shared CMPE 295A B Folder/Assignments/CMPE 295B Assignments/Final ML Model/'

with open(f'{file_path}traffic_mlp_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)
print("Model loaded successfully")

Model loaded successfully


In [ ]:
# Load the CSV file, handling potential issues with quotes and line breaks
network_data = pd.read_csv('/content/network_data_window.csv', on_bad_lines='skip')

In [ ]:
network_data.head()

,ip.src,ip.dst,tcp.srcport,frame.protocols,frame.time_relative,ip.ttl,frame.len,tcp.seq_raw,tcp.ack_raw,tcp.window_size
0,59.166.0.3,149.171.126.0,1972.0,sll:ethertype:ip:tcp,0.000000,32.0,68,747668635.0,751906714.0,65160.0
1,59.166.0.3,149.171.126.0,1972.0,sll:ethertype:ip:tcp,0.000007,31.0,68,747668635.0,751906714.0,65160.0
2,149.171.126.0,59.166.0.3,6881.0,sll:ethertype:ip:tcp:bittorrent,0.000119,30.0,537,751906714.0,747668635.0,26064.0
3,149.171.126.0,59.166.0.3,6881.0,sll:ethertype:ip:tcp,0.000121,29.0,537,751906714.0,747668635.0,26064.0
4,59.166.0.3,149.171.126.0,1972.0,sll:ethertype:ip:tcp,0.000418,32.0,68,747668635.0,751907183.0,65160.0


In [ ]:
network_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186378 entries, 0 to 1186377
Data columns (total 10 columns):
 #   Column               Non-Null Count    Dtype  
---  ------               --------------    -----  
 0   ip.src               1186210 non-null  object 
 1   ip.dst               1186210 non-null  object 
 2   tcp.srcport          1167416 non-null  float64
 3   frame.protocols      1186378 non-null  object 
 4   frame.time_relative  1186378 non-null  float64
 5   ip.ttl               1186210 non-null  float64
 6   frame.len            1186378 non-null  int64  
 7   tcp.seq_raw          1167416 non-null  float64
 8   tcp.ack_raw          1167416 non-null  float64
 9   tcp.window_size      1167416 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 90.5+ MB


In [ ]:
network_data.dropna(subset=["frame.time_relative", "tcp.srcport", "ip.ttl", "frame.len"], inplace=True)

In [ ]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186378 entries, 0 to 1186377
Data columns (total 15 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   is_source  1186378 non-null  bool   
 1   dur        0 non-null        float64
 2   sttl       1186378 non-null  object 
 3   dttl       1186210 non-null  object 
 4   sload      1186378 non-null  object 
 5   dload      1186378 non-null  object 
 6   sloss      1186378 non-null  int64  
 7   dloss      1186378 non-null  int64  
 8   sjit       1186378 non-null  float64
 9   djit       1186378 non-null  float64
 10  swin       1186378 non-null  object 
 11  dwin       1167416 non-null  object 
 12  stcpb      1186378 non-null  object 
 13  dtcpb      1167416 non-null  object 
 14  spkts_log  1186378 non-null  float64
dtypes: bool(1), float64(4), int64(2), object(8)
memory usage: 127.9+ MB


In [ ]:
processed_data = pd.DataFrame()


network_data['frame.time_relative'] = pd.to_numeric(network_data['frame.time_relative'], errors='coerce') # Convert 'frame.time_relative' to numeric
processed_data['is_source'] = network_data['ip.src'] == '59.166.0.3'
processed_data['dur'] = network_data['frame.time_relative']

processed_data['sttl'] = np.where(processed_data['is_source'], network_data['ip.ttl'], 0)
processed_data['dttl'] = np.where(~processed_data['is_source'], network_data['ip.ttl'], 0)

processed_data['sload'] = np.where(processed_data['is_source'], network_data['frame.len'], 0)
processed_data['dload'] = np.where(~processed_data['is_source'], network_data['frame.len'], 0)

processed_data['sloss'] = 0  # Placeholder, needs retransmission data
processed_data['dloss'] = 0  # Placeholder

processed_data['sjit'] = np.where(processed_data['is_source'], network_data['frame.time_relative'].diff().fillna(0), 0)
processed_data['djit'] = np.where(~processed_data['is_source'], network_data['frame.time_relative'].diff().fillna(0), 0)

processed_data['swin'] = np.where(processed_data['is_source'], network_data['tcp.window_size'], 0)
processed_data['dwin'] = np.where(~processed_data['is_source'], network_data['tcp.window_size'], 0)

processed_data['stcpb'] = np.where(processed_data['is_source'], network_data['tcp.seq_raw'], 0)
processed_data['dtcpb'] = np.where(~processed_data['is_source'], network_data['tcp.ack_raw'], 0)

# Log-transformed counts for packets and bytes, convert the argument to float before applying log1p
processed_data['spkts_log'] = np.log1p(np.where(processed_data['is_source'], network_data['frame.len'], 0).astype(float))
processed_data['dpkts_log'] = np.log1p(np.where(~processed_data['is_source'], network_data['frame.len'], 0).astype(float))

processed_data['sttl'] = network_data['ip.ttl']
processed_data['dttl'] = network_data['ip.ttl']

processed_data['sload'] = np.where(processed_data['is_source'], network_data['frame.len'], 0)
processed_data['dload'] = np.where(~processed_data['is_source'], network_data['frame.len'], 0)

processed_data['sloss'] = 0  # Placeholder
processed_data['dloss'] = 0  # Placeholder
processed_data['sjit'] = network_data['frame.time_relative'].diff().fillna(0)
processed_data['djit'] = processed_data['sjit']  # Placeholder
processed_data['swin'] = network_data['tcp.window_size']
processed_data['dwin'] = network_data['tcp.window_size']
processed_data['stcpb'] = network_data['tcp.seq_raw']
processed_data['dtcpb'] = network_data['tcp.ack_raw']
processed_data['tcprtt'] = 0.0  # Placeholder
processed_data['synack'] = 0.0  # Placeholder
processed_data['ackdat'] = 0.0  # Placeholder

# Compute mean packet sizes based on source or destination
processed_data['smean'] = np.where(processed_data['is_source'], network_data['frame.len'], 0)
processed_data['dmean'] = np.where(~processed_data['is_source'], network_data['frame.len'], 0)

processed_data['spkts_log'] = np.log1p(processed_data['smean'])
processed_data['dpkts_log'] = np.log1p(processed_data['dmean'])
processed_data['sbytes_log'] = np.log1p(processed_data['sload'])
processed_data['dbytes_log'] = np.log1p(processed_data['dload'])
processed_data['sinpkt_log'] = np.log1p(processed_data['sjit'])
processed_data['dinpkt_log'] = np.log1p(processed_data['djit'])

# One-hot encode protocols
protocols = network_data['frame.protocols'].str.get_dummies(sep=':').iloc[:, :10]
processed_data = pd.concat([processed_data, protocols], axis=1)

# Rename protocol columns to proto_1, proto_2, etc.
num_protocols = protocols.shape[1]  # Get the number of protocol columns
new_protocol_names = [f'proto_{i+1}' for i in range(num_protocols)]
processed_data.rename(columns=dict(zip(protocols.columns, new_protocol_names)), inplace=True)

In [ ]:
processed_data.to_csv('processed_data.csv', index=False)

In [ ]:
processed_data.head()

,is_source,dur,sttl,dttl,sload,dload,sloss,dloss,sjit,djit,...,proto_1,proto_2,proto_3,proto_4,proto_5,proto_6,proto_7,proto_8,proto_9,proto_10
0,True,0.000000,32.0,32.0,68,0,0,0,0.000000,0.000000,...,0,0,0,0,0,0,1,0,0,0
1,True,0.000007,31.0,31.0,68,0,0,0,0.000007,0.000007,...,0,0,0,0,0,0,1,0,0,0
2,False,0.000119,30.0,30.0,0,537,0,0,0.000112,0.000112,...,1,0,0,0,0,0,1,0,0,0
3,False,0.000121,29.0,29.0,0,537,0,0,0.000002,0.000002,...,0,0,0,0,0,0,1,0,0,0
4,True,0.000418,32.0,32.0,68,0,0,0,0.000297,0.000297,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
processed_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1167416 entries, 0 to 1186377
Data columns (total 35 columns):
 #   Column      Non-Null Count    Dtype  
---  ------      --------------    -----  
 0   is_source   1167416 non-null  bool   
 1   dur         1167416 non-null  float64
 2   sttl        1167416 non-null  float64
 3   dttl        1167416 non-null  float64
 4   sload       1167416 non-null  int64  
 5   dload       1167416 non-null  int64  
 6   sloss       1167416 non-null  int64  
 7   dloss       1167416 non-null  int64  
 8   sjit        1167416 non-null  float64
 9   djit        1167416 non-null  float64
 10  swin        1167416 non-null  float64
 11  dwin        1167416 non-null  float64
 12  stcpb       1167416 non-null  float64
 13  dtcpb       1167416 non-null  float64
 14  spkts_log   1167416 non-null  float64
 15  dpkts_log   1167416 non-null  float64
 16  tcprtt      1167416 non-null  float64
 17  synack      1167416 non-null  float64
 18  ackdat      1167416 non-nul

In [ ]:
training_features

array(['dur', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss', 'sjit',
       'djit', 'swin', 'stcpb', 'dtcpb', 'dwin', 'tcprtt', 'synack',
       'ackdat', 'smean', 'dmean', 'spkts_log', 'dpkts_log', 'sbytes_log',
       'dbytes_log', 'sinpkt_log', 'dinpkt_log', 'proto_1', 'proto_2',
       'proto_3', 'proto_4', 'proto_5', 'proto_6', 'proto_7', 'proto_8',
       'proto_9', 'proto_10'], dtype=object)

In [ ]:
# Test the loaded model
# sample_features = processed_data.head().drop(columns=['is_source'])
# Get the feature names used during training
training_features = loaded_model.feature_names_in_

# Ensure sample_features has the same columns and order as training data
sample_features = processed_data.head()[training_features]

print("Sample features:")
print(sample_features)

prediction = loaded_model.predict(sample_features)
print("Prediction:", prediction)

Sample features:
        dur  sttl  dttl  sload  dload  sloss  dloss      sjit      djit  \
0  0.000000  32.0  32.0     68      0      0      0  0.000000  0.000000   
1  0.000007  31.0  31.0     68      0      0      0  0.000007  0.000007   
2  0.000119  30.0  30.0      0    537      0      0  0.000112  0.000112   
3  0.000121  29.0  29.0      0    537      0      0  0.000002  0.000002   
4  0.000418  32.0  32.0     68      0      0      0  0.000297  0.000297   

      swin  ...  proto_1  proto_2  proto_3  proto_4  proto_5  proto_6  \
0  65160.0  ...        0        0        0        0        0        0   
1  65160.0  ...        0        0        0        0        0        0   
2  26064.0  ...        1        0        0        0        0        0   
3  26064.0  ...        0        0        0        0        0        0   
4  65160.0  ...        0        0        0        0        0        0   

   proto_7  proto_8  proto_9  proto_10  
0        1        0        0         0  
1        1 